# Geospatial Data Analysis I 

## Spatial Interploation - Exercise

### Part 1: Deterministic Interpolation

There are several Python Packages that contain useful libraries and functions for spatial interpolation, e.g.: 

- scipy.interpolate: Univariate & multivariate interpolation, functions for linear, spline, Nearest Neighbour und radial basis function interpolation

- scipy.spatial: Delauney Tesselation, Voronoi Diagrams und several plotting functions

- Inverse distance weighting: e.g. via PyGeM (https://mathlab.github.io/PyGeM/index.html), or other packages (https://github.com/paulbrodersen/inverse_distance_weighting)

In this exercise we are going to take a closer look at the options in `scipy`. As data we are going to use measured groundwater heads and temperatures in Karlsruhe (Menberg et al., 2013). 

#### Exercise 1: scipy.spatial and scipy.interpolate.griddata

For interpolating any data we first need to specifiy a regular grid specifying the points on which to interpolate.  

- First, read to the data from "interp_data.csv" as a Pandas DataFrame.  

- For defining the interpolation grid, start with extracting the minimum (`data.min`) and maximum (`data.max`) values of the columns with the x and y coordinates. In order to create a small buffer around the extreme points add / substract a small value (e.g. 1000m) from the min and max values. 

- Then, combine these variables to a list that specfies the overall extent of the grid: `extent = x_min, x_max, y_min, y_max` 

- Last, create one array for x and y coordinates each using the function `x, y = np.mgrid[xmin:xmax:xstep, ymin:ymax:ystep]`. Finally, plot the created coordinates as a 2D grid (`plt.scatter()`) to visualise the interpolation grid.

In [ ]:
# [1]


Now, we can interpolate the groundwater temperature data on the created grid using *scipy* and `scipy.interpolate.griddata()`. This function supports different methods (Nearest Neighbour, linear interpolation, cubic spline interpolation, etc.), which we want to compara against each other. 

- First, we need to create tuples from the orginal x and y coordinates, i.e. combine them into a single entry of an entry indicated by a double bracket (`index [[x, y]]`). To extract the numeric values from the DataFrame you can use `data.values`, or combined to `data[[value1, value2]].values`. 

- Next, create the interpolated data sets using `scipy.interpolate.griddata()`, and as arguments the created tuples, the values of the measured groundwater temperatures (`data.values`), the x and y coordinates of the grid as a list (`(x, y)`), and the interpolation method to use (`method=...`). Create three data sets using "nearest", "linear" and " cubic" as inputs for the method argument. 

- Finally, visualise the three interpolation results in a figure with three subplots. You can visualise the interpolated raster data using `matplotlib.pyplot.imshow()` with the interpolated data (transposed as `data.T`), the starting point (`origin='lower'`) and the extent of the data (`extent=area`) as input arguments. As area you can use the defined boundaries of the grid from above. 

- Compare the plotted results. Which method seems to produce the most reasonable results? 

In [ ]:
# [2]


#### Exercise 2: scipy.interpolate.rbf

Besides the options from above, *Scipy* also contains different functions for interpolation with radial basis functions (RBFs). The default option in `scipy.interpolate.Rbf()` is a multi-quadratic function that exactly represents the values at the measurement locations (*exact interpolation*). 

- First, define an *interpolation object* using `scipy.interpolate.Rbf()` using the x and y coordinates of the measured data, as well as the measured values as input arguments. 

- Apply this interpolation object to the x and y coordinates of the created grid (`object(x-grid, y-grid)`), and visualise the resulting interpolated data using `matplotlib.pyplot.imshow()`. 

- In order to compare the interpolated data to the measured values, also plot the measured data using `matplotlib.pyplot.scatter()` setting the groundwater temperature values as color (`c=data.GWTvalues`). 

In [ ]:
# [3]



### Part 2: Stochastic interpolation - Kriging

There are several packages that offer options for interpolation with Kriging:  
 
 - SciKit GStat: several options to analyse variograms, but interpolation only for Ordinary Kriging (i.e. data without trend or drift): https://scikit-gstat.readthedocs.io/en/latest/install.html#conda-forge
 
 - PyKrige: contains implementations for Ordinary and Universal Kriging (data with trend or drift), but less sophisticated options for detailed analysis of variograms https://geostat-framework.readthedocs.io/projects/pykrige/en/stable/index.html

 
In this exercise we are going to use SciKit GStat. Download and install the package in your virtual environment using the command `conda install -c conda-forge scikit-gstat` in the terminal. 

Kriging interpolation always starts with a variogram analysis. 

- First, create a *variogram object* using `skgstat.Variogram()`. As input arguments the functions requires the coordinates of the measured data as tuples (just as in #2), as well as the measured groundwater temperature values.  

- Then, you can visualise the variogram using `VariogramObjekt.plot();` (the ';' is important!). Depending on your laptop this can take a few seconds...

In [ ]:
# [4]


As one can see in the semivariogram plot, the values beyond a lag of 8000 meters is not well supported by data (dropping bar heights in the top row). The variogram analysis should thus be adapted to account for this. 

- Change the maximum lag of the variogram analysis by adding `.maxlag = 8000` to your variogram object. 

- Also, as we have more than 90 measurements we can increase the number of lag classes to 15 by adding `.n_lags=15` to the variogram object. 

- Finally, plot the variogram again as above. 

In [ ]:
# [5]


While the fit of the line looks good for higher lags, this is not the case for short distances (i.e. a lag smaller than 4000). This range however is very important for determining the weights for kriging interpolation. If not defined otherwise `skgstat.Variogram()` by default uses a spherical variance model. 

- In addition to the variogram object above created above (i.e. the one aith a spherical model), create two more with other models. To do so, you can specify the model by `VariogramObject.model = 'spherical'`, or "exponential" and "gaussian".  

- The variogram object also has a functionality to calculate the goodness fit as the root mean square error (RMSE). Use `VariogramObject.rmse` to evaluatethe goodness of fit of all three models. Which one shows the best fit? Also, compare the visual goodness of fit to the RMSE values. 

In [ ]:
# [6]


Pick the variogram model that fits best and perform Ordinary Kriging using `skgstat.OrdinaryKriging()`. 

- First, define an interpolation object using `OrdinaryKriging()`. Required input arguments are the Variogram object (with the best fitting model), as well as the minimum and maximum number of ssurrunding measurement points to be used for interpolation. These numbers depend on the number of measurements and are often determined iteratively. Here, set `min_points=2` and `max_points=8`.  

- Define a regular grid of interpolation points, as in Part 1.  

- Apply the `.transform()` fucntion to the interpolation object. This requires the x and y coordinates as individual 1D Numpy arrays. You can create those arrays by `array.flatten()`. 

- Visualise the interpolation results with `plt.imshow()` . To do so, you have to reshape the 1D array with the interpolated data into a 2D matrix, e.g. with `array.reshape(dimensions)`. You can pick the dimensions from the original grid by `grid.shape`. 

- Also, plot the loaction of the measurements (e.g. as black circles) on top of the interpolated raster. 

In [ ]:
# [7]


## END

### References

Menberg et al. (2013): Long-term evolution of anthropogenic heat fluxes into a subsurface urban heat island, Environ. Sci. Technol. 47(17) (2013) 9747-9755

https://colab.research.google.com/github/agile-geoscience/xlines/blob/master/notebooks/11_Gridding_map_data.ipynb

https://mmaelicke.github.io/scikit-gstat/tutorials/01_getting_started.html